In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-uas-rfm/UAS_RFM - RFM.csv
/kaggle/input/dataset-rfm/UAS_RFM - RFM.csv


In [83]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [84]:
datatrx = pd.read_csv('../input/dataset-rfm/UAS_RFM - RFM.csv', parse_dates=['TrxDate'])
datatrx.head()

,CardID,TrxDate,Amount
0,1000152,2021-06-22,649208
1,1000158,2021-02-15,504063
2,1000156,2021-04-20,817290
3,1000115,2021-09-03,763471
4,1000164,2021-07-04,443094


In [85]:
datatrx.head(10)

,CardID,TrxDate,Amount
0,1000152,2021-06-22,649208
1,1000158,2021-02-15,504063
2,1000156,2021-04-20,817290
3,1000115,2021-09-03,763471
4,1000164,2021-07-04,443094
5,1000136,2021-06-06,219345
6,1000147,2021-04-23,641383
7,1000118,2021-06-06,514425
8,1000167,2021-04-30,849509
9,1000160,2021-10-04,293554


In [86]:
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [87]:
print(datatrx['TrxDate'].min(), datatrx['TrxDate'].max())

2021-01-02 00:00:00 2021-12-02 00:00:00


In [88]:
sd = dt.datetime(2021,12,21)
datatrx['hist']=sd - datatrx['TrxDate']
datatrx['hist'].astype('timedelta64[D]')
datatrx['hist']=datatrx['hist'] / np.timedelta64(1, 'D')
datatrx.head(10)

,CardID,TrxDate,Amount,hist
0,1000152,2021-06-22,649208,182.0
1,1000158,2021-02-15,504063,309.0
2,1000156,2021-04-20,817290,245.0
3,1000115,2021-09-03,763471,109.0
4,1000164,2021-07-04,443094,170.0
5,1000136,2021-06-06,219345,198.0
6,1000147,2021-04-23,641383,242.0
7,1000118,2021-06-06,514425,198.0
8,1000167,2021-04-30,849509,235.0
9,1000160,2021-10-04,293554,78.0


In [89]:
datatrx=datatrx[datatrx['hist'] < 730]
datatrx.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.8 KB


In [90]:
rfmTable = datatrx.groupby('CardID').agg({'hist': lambda x:x.max(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(10)

,recency,frequency,monetary_value
CardID,,,
1000101,209.0,1,256626
1000102,291.0,3,1966664
1000103,188.0,1,105677
1000104,349.0,2,823419
1000105,311.0,5,2301003
1000106,238.0,2,1568182
1000107,278.0,3,586382
1000108,334.0,3,913041
1000109,208.0,2,942097


In [91]:
quartiles = rfmTable.quantile(q=[0.20,0.30,0.40])
print(quartiles, type(quartiles))

     recency  frequency  monetary_value
0.2    231.0        1.0        587496.0
0.3    243.0        2.0        755493.0
0.4    273.0        2.0        942097.0 <class 'pandas.core.frame.DataFrame'>


In [92]:
def RClass(x,p,d):
    if x <= d[p][0.20]:
        return 4
    elif x <= d[p][0.30]:
        return 3
    elif x <= d[p][0.40]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.30]:
        return 2
    elif x <= d[p][0.40]: 
        return 3
    else:
        return 4

In [93]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

In [95]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,209.0,1,256626,4,1,1,411
1000102,291.0,3,1966664,1,4,4,144
1000103,188.0,1,105677,4,1,1,411
1000104,349.0,2,823419,1,2,3,123
1000105,311.0,5,2301003,1,4,4,144
1000106,238.0,2,1568182,3,2,4,324
1000107,278.0,3,586382,1,4,1,141
1000108,334.0,3,913041,1,4,3,143
1000109,208.0,2,942097,4,2,3,423


In [97]:
print("Pelanggan setia: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('konsumen: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("banyak pelanggan: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("resiko: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('hampir kehilangan: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('kehilangan pelanggan: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('kehilangan pelanggan ekonomis: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Pelanggan setia:  1
konsumen:  36
banyak pelanggan:  42
resiko:  42
hampir kehilangan:  2
kehilangan pelanggan:  1
kehilangan pelanggan ekonomis:  2
